# Produce messages to an Apache Kafka topic

In this notebook you will learn how to produce messages to an Apache Kafka topic

![Produce messages to an Apache Kafka Topic](../img/produce.png)

---

## Prerequisites

To start the tutorial you need to:

* Download the Aiven for Apache Kafka SSL certificates as mentioned in the [README instructions](../README.md#Download-the-required-SSL-certificates)
* Retrieve the Aiven for Apache Kafka hostname and port, from the [Aiven Console](https://console.aiven.io/) in the Aiven for Apache Kafka service overview
* Substitute the Apache Kafka hostname and port in the parameters below

In [ ]:
# Replace the following two placeholders with Aiven for Apache Kafka service hostname and port

hostname="<MYKAFKAHOST>"
port="<MYKAFKAPORT>"

---

### Install the required libraries

The following installs the [kafka-python library](https://kafka-python.readthedocs.io/en/master/) we'll use for the tutorial

In [ ]:
!pip install confluent-kafka

---

## Create an Apache Kafka producer

The next step is to configure the Kafka producer.

In [ ]:
import json
from confluent_kafka import SerializingProducer

def json_serializer(msg, s_obj):
    return json.dumps(msg).encode('ascii')

conf = {
    'bootstrap.servers': hostname+":"+port,
    'client.id': 'myclient',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.serializer': json_serializer,
    'key.serializer': json_serializer
    }

producer = SerializingProducer(conf)

---
## Push your first message to the `weather` topic

In the below section we're pushing the first message to the `weather` topic.

> Note that we're producing data to a topic named `weather`, you need to have the topic created beforehand in Apache Kafka or the `kafka.auto_create_topics_enable` parameter enabled

The `flush` function ensures that the message is sent to Kafka

In [ ]:
import requests
import uuid
import time
api_url = "https://api.openweathermap.org/data/2.5/weather?zip=80550,us&&units=imperial&appid=66ba4d476af08513ac4cb2b17666e5f2"
# This is an API request to pull weather based on US zip code. Feel free to update it to another zip code!

topic = "weather"

for _ in range(60):  # Produce 60 events
    key = str(uuid.uuid4())
    response = requests.get(api_url)
    payload = {
        'id': key,
        'event_type': 'current_weather',
        'timeupdate': time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime()),
    }
    payload.update(response.json())
    payload.update(id=key)
    producer.produce(topic, key=key, value=payload)
    time.sleep(60) # one event per minute


producer.flush()